In [ ]:
from IPython.core.display import HTML
css = open('../data/style-table.css').read() + open('../data/style-notebook.css').read()
HTML('../data/<style>{}</style>'.format(css))

In [ ]:
%config IPCompleter.greedy = True

In [ ]:
import datetime, os


from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

from journalfiles import JournalFiles
import pandas as pd
from structjour.pandasutil import DataFrameUtil, InputDataFrame, ToCSV_Ticket as Ticket
from structjour.tradeutil import ReqCol, FinReqCol, XLImage, TradeUtil
#TradeUtil, FinReqCol, ReqCol, 

jf=JournalFiles(theDate=datetime.date(2018, 9, 21),infile = 'trades.csv', outdir = 'out/', mydevel=True)
jf._printValues()
        
tkt = Ticket(jf)

tu = TradeUtil()
trades, jf =tkt.newDFSingleTxPerTicket()
# trades = pd.read_csv(jf.inpathfile)

idf = InputDataFrame()
reqCol = ReqCol()
finalReqCol = FinReqCol()

DataFrameUtil.checkRequiredInputFields(trades, reqCol.columns)
trades = idf.zeroPadTimeStr(trades)
trades = trades.sort_values([reqCol.acct, reqCol.ticker, reqCol.time])
trades = idf.mkShortsNegative(trades)
swingTrade = idf.getOvernightTrades(trades)
swingTrade = idf.figureOvernightTransactions(trades)
trades = idf.insertOvernightRow(trades,swingTrade)

trades = tu.addFinReqCol(trades)
newTrades = trades[finalReqCol.columns]
newTrades.copy()
nt = newTrades.sort_values([finalReqCol.ticker,finalReqCol.acct,  finalReqCol.time])

In [ ]:
nt.head()

In [ ]:
nt = tu.writeShareBalance(nt)
nt.head()

In [ ]:
nt = tu.addStartTime(nt)
nt.head()

In [ ]:
nt = nt.sort_values([finalReqCol.start, finalReqCol.acct, finalReqCol.time])
nt.head()

In [ ]:
nt = tu.addTradeIndex(nt)
nt.head()


In [ ]:
nt = tu.addTradePL(nt)
nt.head()

In [ ]:
nt = tu.addTradeDuration(nt)
nt.head()

In [ ]:
nt = tu.addTradeName(nt)
nt.head()

In [ ]:
nt=DataFrameUtil.addRows(nt,1)
nt.head()

In [ ]:
nt = tu.addSummaryPL(nt)
nt.head()

In [ ]:
    
ldf=tu.getTradeList(nt)

In [ ]:
ldf[3]
df=ldf[3]

In [ ]:
from thetradeobject import SumReqFields

from thetradeobject import SumReqFields, TheTradeObject
import pandas as pd

srf = SumReqFields()


tto = TheTradeObject(df)

In [ ]:
tto.runSummary()[['Strategy', 'EntryHead', 'Entry1', 'Entry2', 'ExitHead', 'Exit1', 'Exit2', 
                  'TargHead', 'Target', 'TargDiff', 'StopHead', 'StopLoss', 'SLDiff', 
                  'RRHead', 'RR', 'MaxHead', 'MaxLoss', 'Explain', 'Notes']]

In [ ]:
srf.columns



### Note that the interview stuff will be taken care of during the image insertion allowing the user to do all the review of a tradeafter viewing the chart. The interview stuff will include: 1. strategy  2. brief strategy note  3. target  4. stop loss  5. description of trade.  6. Notes and analysis of the trade

### On Second thought, the strategy note, trade explanation and analysis will be left to do in excel. The Gui Version os structjour will implement the review. Otherwise I would have to include some facility for entering multiple sentences in the console app.  Not that difficult but not that helpful when it can be done in Excel wilh all the visuals present.